In [2]:
%matplotlib inline
import cv2
from matplotlib import pyplot  as plt
import numpy as np
from math import cos 
from PIL import Image as _Image
from skimage.filters import (threshold_otsu, threshold_niblack,
                             threshold_sauvola)
from skimage.transform import integral_image
from scipy import ndimage
from skimage import img_as_ubyte
from math import *

In [3]:
images = [cv2.imread('/home/shanu/Desktop/WhatsAppImage1.jpeg') , cv2.imread('/home/shanu/Desktop/WhatsAppImage2.jpeg')]

In [4]:
def printImage(image,imageName):
    cv2.imshow(imageName,image)
    return cv2.waitKey(0)

In [5]:
def rescaling(image):
    max_dimension = max(image.shape)
    scale= 640.0/max_dimension
    image = cv2.resize(image,None,fx=scale,fy=scale,interpolation = cv2.INTER_AREA)
    return image

In [6]:
image = images[0]
image = rescaling(image)
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
# integral = integral_image(gray)
thresh_sauvola = threshold_sauvola(gray, window_size=15, k=0.3,r=128)
sauvola_binary = gray > thresh_sauvola
sauvola_binary = np.uint8(sauvola_binary)
plt.imsave('Sauvola',np.uint8(sauvola_binary)*255,cmap = 'gray',format = 'png')
img = plt.imread('/home/shanu/Sauvola')

In [50]:
def checkRatio(state_size,totalFinderSize):
    if totalFinderSize<7 or totalFinderSize ==0:
        return False

    moduleSize = ceil(totalFinderSize / 7.0)
    maxVariance = moduleSize/2

    retVal= ((abs(moduleSize - (state_size[0])) < maxVariance) and
        (abs(moduleSize - (state_size[1])) < maxVariance) and
        (abs(3*moduleSize - (state_size[2])) < 3*maxVariance) and
        (abs(moduleSize - (state_size[3])) < maxVariance) and
        (abs(moduleSize - (state_size[4])) < maxVariance))

    return retVal

In [51]:
img_mat = sauvola_binary
skip_rows = 3
rows = img_mat.shape[0]
cols = img_mat.shape[1]
PossibleCentres = []
estimatedModuleSize = []
for r in range(skip_rows-1,rows,skip_rows):
    stateCount = [0]*5
    currentState = 0
    
    for c in range(0,cols):
#         pattern black:white:black:white:black::1:1:3:1:1
        if img_mat[r][c] ==0 :
            if((currentState & 0x1)==1):
                print 'currentState',currentState
                print 'stateCount',stateCount
                currentState = currentState+1
            #for both W->B and B->B transition     
            stateCount[currentState] = stateCount[currentState]+1  
        else:
            if((currentState & 0x1)==1):
#                     W->W transition
                stateCount[currentState] = stateCount[currentState]+1
            else:
#               but, we were counting black pixels
                if currentState==4:
#                    We found the 'white' area AFTER the finder patter
#                    Do processing for it here
                    stateCountTotal = 0   
                    for count in stateCount:
                        stateCountTotal = stateCountTotal+count
                    if(checkRatio(stateCount,stateCountTotal)):
                            #horizontal vertical diagonal checks
                        confirmed = handlePossibleCenter(img_mat, stateCount, r, c,stateCountTotal)
                    else:
                        currentState = 3
                        stateCount[0] = stateCount[2]
                        stateCount[1] = stateCount[3]
                        stateCount[2] = stateCount[4]
                        stateCount[3] = 1
                        stateCount[4] = 0
                        continue    
                    
                    #reset currentState and stateCount       
                    currentState = 0
                    stateCount = [0 for i in stateCount] 
                        
                else:
#                   increment the state B->W transition
                    print 'currentState', currentState
                    print 'stateCount',stateCount
                    currentState = currentState+1
                    stateCount[currentState] = stateCount[currentState]+1
        
  

currentState 0
stateCount [16, 0, 0, 0, 0]
currentState 1
stateCount [16, 10, 0, 0, 0]
currentState 2
stateCount [16, 10, 4, 0, 0]
currentState 3
stateCount [16, 10, 4, 9, 0]
currentState 3
stateCount [4, 9, 1, 3, 0]
currentState 3
stateCount [1, 3, 1, 9, 0]
currentState 3
stateCount [1, 9, 1, 17, 0]
currentState 3
stateCount [1, 17, 3, 13, 0]
currentState 3
stateCount [3, 13, 1, 11, 0]
currentState 3
stateCount [1, 11, 1, 31, 0]
currentState 3
stateCount [1, 31, 3, 7, 0]
currentState 3
stateCount [3, 7, 8, 3, 0]
currentState 3
stateCount [8, 3, 1, 13, 0]
currentState 3
stateCount [1, 13, 8, 159, 0]
currentState 3
stateCount [8, 159, 7, 11, 0]
currentState 3
stateCount [7, 11, 9, 10, 0]
currentState 3
stateCount [9, 10, 1, 3, 0]
currentState 0
stateCount [10, 0, 0, 0, 0]
currentState 1
stateCount [10, 14, 0, 0, 0]
currentState 2
stateCount [10, 14, 9, 0, 0]
currentState 3
stateCount [10, 14, 9, 36, 0]
currentState 3
stateCount [9, 36, 4, 8, 0]
currentState 3
stateCount [4, 8, 21, 28, 0

NameError: name 'handlePossibleCenter' is not defined

In [48]:
# d = []
# A = np.array([12,13])
# B = np.array([14,16])
# C = A-B
# print type(C)
# d.append(C.tolist())
# print type(d[0])
# D = np.dot(C,C)

# d.append(np.vstack([C,([D,D])]).tolist())
# print d

# for a in C:
#     print a
stateCount = [5]*5
stateCount=[0 for i in stateCount]
print stateCount

[0, 0, 0, 0, 0]


In [ ]:
 def centerFromEnd(stateCountTotal,col):
    return float(col)-float(stateCountTotal/2)


 def crossCheckVertical(img, startRow,centerCol,centralCount,stateCountTotal) :
    maxRows = img.shape[0]
    crossCheckStateCount = [0]*5
    row = startRow
    while(row>=0 and img[row][centerCol]==0) :
        crossCheckStateCount[2] = crossCheckStateCount[2]+1
        row = row-1

    if row<0: 
        return nan

    while(row>=0 and img[row][centerCol] ==1 and crossCheckStateCount[1]<centralCount) :
        crossCheckStateCount[1] = crossCheckStateCount[1]+1
        row = row-1

    if row<0 or crossCheckStateCount[1]>=centralCount :
        return nan

    while row>=0 and img[row][centerCol]==0 and crossCheckStateCount[0]<centralCount :
        crossCheckStateCount[0] = crossCheckStateCount[0]+1
        row = row-1

    if(row<0 or crossCheckStateCount[0]>=centralCount) :
        return nan
    
#      Now we traverse down the center
    row = startRow+1
    while(row<maxRows and img[row][centerCol]==0): 
        crossCheckStateCount[2] = crossCheckStateCount[2]+1
        row = row+1

    if(row==maxRows) :
        return nan
    

    while(row<maxRows and img[row][centerCol]==1 and crossCheckStateCount[3]<centralCount):
        crossCheckStateCount[3] = crossCheckStateCount[3]+1
        row = row+1
    
    if(row==maxRows or crossCheckStateCount[3]>=stateCountTotal): 
        return nan

    while(row<maxRows and img[row][centerCol]==0 and crossCheckStateCount[4]<centralCount) :
        crossCheckStateCount[4]= crossCheckStateCount[4]+1
        row = row+1
    
    if(row==maxRows or crossCheckStateCount[4]>=centralCount) :
        return nan
 
    crossCheckStateCountTotal = stateCountTotal
    
    if(5*abs(crossCheckStateCountTotal-stateCountTotal) >= 2*stateCountTotal):
        return nan

    center = centerFromEnd(crossCheckStateCountTotal, row)
    return center if checkRatio(crossCheckStateCount) else nan 
   
 
 def crossCheckHorizontal(img, centerRow, startCol, centerCount, stateCountTotal) :
    maxCols = img.shape[1]
    stateCount = [0]*5

    col = startCol
    while(col>=0 and img[centerRow][col]==0) :
        stateCount[2] = stateCount[2]+1
        col = col-1

    if(col<0) :
        return nan    

    while(col>=0 and img[centerRow][col]==1 and stateCount[1]<centerCount):
        stateCount[1] = stateCount[1]+1
        col = col-1
        
    if(col<0 or stateCount[1]==centerCount) :
        return nan

    while(col>=0 and img[centerRow][col]==1 and stateCount[0]<centerCount):
        stateCount[0] = stateCount[0]+1
        col = col-1
    
    if(col<0 or stateCount[0]==centerCount) :
        return nan
    
    col = startCol + 1
    
    while(col<maxCols and img[centerRow][col]==0): 
        stateCount[2] = stateCount[2]+1
        col = col+1

    if(col==maxCols):
        return nan

    while(col<maxCols and img[centerRow][col]==1 and stateCount[3]<centerCount) :
        stateCount[3] = stateCount[3]+1
        col = col+1
     
    if(col==maxCols or stateCount[3]==centerCount) :
        return nan 
     

    while(col<maxCols and img[centerRow][col]==0 and stateCount[4]<centerCount) :
        stateCount[4] = stateCount[4]+1
        col = col+1
     
    if(col==maxCols or stateCount[4]==centerCount) :
        return nan
    newStateCountTotal = stateCountTotal
    
    if(5*abs(stateCountTotal-newStateCountTotal) >= stateCountTotal) :
        return nan

    return centerFromEnd(stateCount,col) if checkRatio(stateCount) else nan

def crossCheckDiagonal(img, centerRow, centerCol, maxCount, stateCountTotal) :
    stateCount = [0]*5

    while(centerRow>=i and centerCol>=i and img[centerRow-i][centerCol-i]==0) :
        stateCount[2] = stateCount[2]+1 
        i = i+1 
     
    if(centerRow<i or centerCol<i) :
        return false 
     

    while(centerRow>=i and centerCol>=i and img[centerRow-i][centerCol-i]==1 and stateCount[1]<=maxCount) :
        stateCount[1] = stateCount[1]+1 
        i = i+1 
     
    if(centerRow<i or centerCol<i or stateCount[1]>maxCount) :
        return false 
     

    while(centerRow>=i and centerCol>=i and img.at<uchar>(centerRow-i, centerCol-i)<128 and stateCount[0]<=maxCount) :
        stateCount[0] = stateCount[0]+1 
        i = i+1 
     
    if(stateCount[0]>maxCount) :
        return false 
     
    
    maxCols = img.shape[1] 
    maxRows = img.shape[0] 
    i=1
    while((centerRow+i)<maxRows and (centerCol+i)<maxCols and img.at<uchar>(centerRow+i, centerCol+i)<128) :
        stateCount[2] = stateCount[2]+1 
        i = i+1 
     
    if((centerRow+i)>=maxRows or (centerCol+i)>=maxCols) :
        return false 
     

    while((centerRow+i)<maxRows and (centerCol+i)<maxCols and img.at<uchar>(centerRow+i, centerCol+i)>=128 and stateCount[3]<maxCount) :
        stateCount[3] = stateCount[3]+1 
        i = i+1 
     
    if((centerRow+i)>=maxRows or (centerCol+i)>=maxCols or stateCount[3]>maxCount) :
        return false 
     

    while((centerRow+i)<maxRows and (centerCol+i)<maxCols and img.at<uchar>(centerRow+i, centerCol+i)<128 and stateCount[4]<maxCount) :
        stateCount[4] = stateCount[4]+1 
        i = i+1 
     
    if((centerRow+i)>=maxRows or (centerCol+i)>=maxCols or stateCount[4]>maxCount) :
        return false 
     
    newStateCountTotal = stateCountTotal

    return (abs(stateCountTotal - newStateCountTotal) < 2*stateCountTotal) and checkRatio(stateCount)

    

In [12]:
def handlePossibleCentres(img, stateCount, row, col,stateCountTotal):
    centreCol = centerFromEnd(stateCountTotal, col)
    centerRow = crossCheckVertical(img, row, centerCol, stateCount[2], stateCountTotal)
    if isnan(centreRow):
        return False
    centerCol = crossCheckHorizontal(img, centerRow, centerCol, stateCount[2], stateCountTotal)
    if(isnan(centerCol)) :
        return false
    validPattern = crossCheckDiagonal(img, centerRow, centerCol, stateCount[2], stateCountTotal)
    if not (validPattern) :
        return false
    ptNew = np.array([centreCol,centreRow])
    newEstimatedModuleSize = stateCountTotal / 7.0f
    found = false
    idx = 0
    for pt in possibleCenters: 
        pt = np.array(pt)
        diff = pt - ptNew
        dist = np.dot(diff,diff)
#       If the distance between two centers is less than 10px, they're the same.
        if(dist < 10) :
            pt = pt + ptNew
            pt = pt/2.0f
            estimatedModuleSize[idx] = (estimatedModuleSize[idx] + newEstimatedModuleSize)/2.0f
            found = True
            break
        idx+1
    if found==False:
        possibleCenters.append(ptNew.tolist())
        estimatedModuleSize.append(newEstimatedModuleSize)
        return True
    return False    
    
    

In [ ]:
Image = images[0]
# print Image
Image_test = rescaling(_Image)
image = Image_test.copy()
hsv = cv2.cvtColor(Image_test, cv2.COLOR_BGR2HSV)
# printImage(Image,"Image")
blue_channel = hsv[:,:,0]
green_channel = hsv[:,:,1]
red_channel = hsv[:,:,2]
thresh = (np.abs(np.subtract(blue_channel,red_channel))+np.abs(np.subtract(blue_channel,green_channel))+np.abs(np.subtract(green_channel,red_channel)))/40

thresh = np.stack((thresh,thresh,thresh),axis = 2)
Image_test[:,:,:] = (thresh>=1)*255
printImage(Image_test,"thresh")
printImage(image,"image")
printImage(Image_test,"test")
cv2.destroyAllWindows()
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
Image_closing = cv2.morpholoelse:gyEx(Image, cv2.MORPH_CLOSE, kernel)
# Image_open = cv2.morphologyEx(Image_closing, cv2.MORPH_OPEN, kernel)
# printImage(Image_closing,"closed")
# cv2.destroyAllWindows()
# hist = cv2.calcHist(Image_closing,[0],None,[256],[0,256])
# plt.plot(hist,'b')
# plt.xlim([0,256])
# plt.show
ret,Image_res = cv2.threshold(Image_closing,10,255,cv2.THRESH_BINARY_INV)
contours,hierarchy_arr  = cv2.findContours(Image_res.copy(),cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
res_image = Image_test+Image_closing
# res_image = cv2.bitwise_and(Image_test,Image_res)
# printImage(res_image,"res_image")
# contours,hierarchy_arr  = cv2.findContours(Image_closing.copy(),cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
# contours = sorted(contours,key = cv2.contourArea,Reverse = True)
# final_image = cv2.drawContours(images[0],[contours],-1,(0,255,0),3)
# printImage(final_image,"thresh_image")
cv2.destroyAllWindows()

In [ ]:
image = images[0]
image = rescaling(image)
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
printImage(hsv,"hsv_image")
cv2.destroyAllWindows()
channels = np.array([0,1])
chans = cv2.split(hsv)
colors = ("b", "g", "r")
black = []
for (chan, color) in zip(chans, colors):
    hist = cv2.calcHist([chan], [0], None, [256], [0, 256])
    black.append(hist.max)
    plt.subplot(321)
    plt.plot(hist,color)
plt.show
#gray scaling, smoothening plotting its histogram from histogram if single mode or bimodal 
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
printImage(gray,"gray")
cv2.destroyAllWindows()
#applying histogram equalization
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(6,6))
equalizedImage = clahe.apply(gray)

# cv2.imwrite('clahe_2.jpg',equalizedImage)
# cv2.imwrite('gray_orig.jpg',gray)
# hist1 = cv2.calcHist([gray], [0], None, [256], [0, 256])
# plt.subplot(322)
# plt.plot(hist1,color)

# hist2 = cv2.calcHist([equalizedImage], [0], None, [256], [0, 256])
# plt.subplot(323)
# plt.plot(hist2,color)
# plt.show



In [ ]:
window_size = 10
plt.figure(figsize=(8,7))
plt.plot(gray[0:0+window_size,0:0+window_size].flatten())
plt.show


In [ ]:
T = np.ones(gray.shape)
window_size = 10
for r in range(0,gray.shape[0] - window_size, window_size):
    for c in range(0,gray.shape[1] - window_size, window_size):
        img = gray[r:window_size,c:window_size]
        t = np.mean(img)*(1+np.std(img,ddof=1)/1240)  
        print t
        T[r:window_size,c:window_size] = T[r:window_size,c:window_size]*t
print T
        


In [ ]:
# try_all_threshold(equalizedImage, figsize=(8, 7), verbose=True)

In [ ]:
# current cell
image = images[0]
# image = cv2.imread('/home/shanu/Desktop/image.jpeg')
image = rescaling(image)
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
print gray.shape
binary_global = gray > threshold_otsu(equalizedImage)
binary_global = binary_global.astype(np.uint8)
plt.imsave('Global',np.uint8(binary_global)*255,cmap = 'gray',format = 'png')

window_size = 25
thresh_niblack = threshold_niblack(gray, window_size=window_size, k=-0.2)
thresh_sauvola = threshold_sauvola(gray, window_size=window_size, k=0.4,r=128)
# printImage(binary_sauvola,'binary_sauvola')
# cv2.destroyAllWindows()
binary_niblack = gray > thresh_niblack
binary_niblack = binary_niblack.astype(np.uint8)
plt.imsave('Niblack',np.uint8(binary_niblack)*255,cmap = 'gray',format = 'png')
binary_sauvola = gray > thresh_sauvola
binary_sauvola = binary_sauvola.astype(np.uint8)
plt.imsave('Sauvola',np.uint8(binary_sauvola)*255,cmap = 'gray',format = 'png')
# binary_sauvola = _Image.fromarray(np.uint8(plt.cm.gray(binary_sauvola)*255))
# print binary_sauvola
# print binary_sauvola

plt.figure(figsize=(8, 7))
plt.subplot(2, 2, 1)
plt.imshow(gray, cmap=plt.cm.gray)
plt.title('Original')
plt.axis('off')

plt.subplot(2, 2, 2)
plt.title('Global Threshold')
plt.imshow(binary_global, cmap=plt.cm.gray)
plt.axis('off')
 
plt.subplot(2, 2, 3)
plt.imshow(binary_niblack, cmap='gray')
plt.title('Niblack Threshold')
plt.axis('off')

plt.subplot(2, 2, 4)
plt.imshow(binary_sauvola, cmap=plt.cm.gray)
plt.title('Sauvola Threshold')
plt.axis('off')
plt.show()
# printImage(binary_sauvola,"binary")
# cv2.destroyAllWindows()
# sampleImage = equalizedImage.ravel()
# print sampleImage.shape
# window = 3*25
# for r in range(0,sampleImage.shape[0]-window,window):
#     image = sampleImage[r:r+window]
#     hist = cv2.calcHist([image], [0], None, [256], [0, 256])
#     normalize = np.divide(hist,np.max(hist))
#     mean = np.mean(normalize)
#     delta = np.std(normalize)
    

In [ ]:
print gray.shape
print thresh_niblack.shape
print binary_niblack, '\n'
print thresh_niblack, '\n'

In [ ]:
#local thersholding
windowsize_r = 80
windowsize_c = 60
# Crop out the window and calculate the histogram
for r in range(0,equ.shape[0] - windowsize_r, windowsize_r):
    for c in range(0,equ.shape[1] - windowsize_c, windowsize_c):
        window = equ[r:r+windowsize_r,c:c+windowsize_c]
        hist = cv2.calcHist([window], [0], None, [256], [0, 256])
        plt.subplot(880+(r/80)*8+(c)/60+1)
        plt.plot(hist)
plt.show


In [ ]:
black = np.uint8([[[0,0,0]]])
black_mask = cv2.cvtColor(black,cv2.COLOR_BGR2HSV)
print black_mask
min_black = np.array([0, 100, 100])
max_black = np.array([0, 255, 255])
Image_ = gray[0]
image_black1 = cv2.inRange(Image_, min_black, max_black)
min_black2 = np.array([170, 100, 100])
max_black2 = np.array([180, 255, 255])
image_black2 = cv2.inRange(Image_, min_black2, max_black2)
image_black1+image_black2

In [ ]:
image1 = rescaling(images[0])
image2 = rescaling(images[1])
gray = [cv2.cvtColor(image1,cv2.COLOR_BGR2GRAY), cv2.cvtColor(image2,cv2.COLOR_BGR2GRAY)]


In [ ]:
for image in gray :
    image.size
    cv2.imshow("image",image)
    cv2.waitKey()
cv2.destroyAllWindows()    

In [ ]:
#plotting histogram
hist = cv2.calcHist(gray[1],[1],None,[256],[0,256])
plt.plot(hist,'b')
plt.xlim([0,256])
plt.show

In [ ]:
for image in gray:
    thresh = otsuThresh(image)
    printImage(thresh,"threshImage")
cv2.destroyAllWindows()    

In [ ]:
image = image[0]
print image.shape

In [ ]:
images  = [cv2.imread('/home/shanu/Desktop/qr-code-pack.jpg',1), cv2.imread('/home/shanu/Desktop/Qr-code-ver-10.png',1), cv2.imread('/home/shanu/Desktop/image.jpeg',1), cv2.imread('/home/shanu/Desktop/Wifi_joiner_QR_code.png',1)]
    

In [ ]:
def grayScale(image,imageName):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    cv2.imshow('gray',gray)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return gray

In [ ]:
def adaptiveThresh(img):
    thresh = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,5,1)
#     printImage(thresh,"adaptiveThresh")
#     cv2.destroyAllWindows()
    return thresh

In [ ]:
def otsuThresh(img): 
#     gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#     printGrayScaled(img)
    blur = cv2.GaussianBlur(img,(7,5),0)
#     printImage(blur)
#     cv2.destroyAllWindows()
    ret2,th2 = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)    # convert to grayscale(binary image) 
    return th2

In [ ]:
#plot histogram
img = image[2]
color = ('b','g','r')
for i,col in enumerate(color):
    histr = cv2.calcHist([img],[i],None,[256],[0,256])
    plt.plot(histr,color = col)
    plt.xlim([0,256])

In [ ]:
# plt.imshow(grayscaled,cmap='gray',interpolation='bicubic')
# plt.show()
def hsv_image(image):
    image_hsv = cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
    cv2.imshow('hsv',image_hsv)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return image_hsv

In [ ]:
#checking binarization method
#draw histogram to check if otsu can be used? o/p - maxing at black and white
# evaluate adaptive thresh check guassianwindow size and input image variation and after image changes like
image = images[3]
#check adaptive binarisation using integral images 
#input is blurred image smoothened image
image = rescaling(image)
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

guassianBlur = cv2.GaussianBlur(gray,(3,3),0)

printImage(guassianBlur,"guassianBlur")
# print guassianBlur


imageHSV = hsv_image(image)
hist1 = cv2.calcHist([gray],[0],None,[256],[0,256])
plt.subplot(221)
plt.plot(hist1)

thresh = adaptiveThresh(guassianBlur)

cv2.destroyAllWindows()
# kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(2,2))
# closing = cv2.morphologyEx(thresh,cv2.MORPH_CLOSE,kernel,iterations = 1)
# printImage(closing, "closing")
# cv2.destroyAllWindows()
#histogram of hsv_image
hist2 = cv2.calcHist([imageHSV],[0],None,[256],[0,256])
plt.subplot(222)
plt.plot(hist2)
#histogram of blurred images
hist3 = cv2.calcHist([guassianBlur],[0],None,[256],[0,256])
plt.subplot(223)
plt.plot(hist3)
plt.show

edged = cv2.Canny(thresh,30,30)
printImage(edged,"edged")
cv2.destroyAllWindows()
# contouredIm,contours,hierarchy = cv2.findContours(thresh.copy(),cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# hierarchy = hierarchy[0]
# contour = np.array(contours)

In [ ]:
image = images[0]
image= rescaling(image)
# printImage(image,"rescaled")
# hsv_image(image)
gray = grayScale(image,"gray") 
median = cv2.medianBlur(gray,5)
printImage(median,"medianBlur")
thresh = otsuThresh(median)
printImage(thresh,"otsuThresh")
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
printImage(closing,"morph")
cv2.destroyAllWindows()

In [ ]:
print len(images)
for x in range(len(images)):
    name = "image" + str(x)
    image = rescaling(images[x])
    gray = grayScale(image,name) 
    image_hsv = hsv_image(image)
    plt.hist(image_hsv.ravel(),256,[0,256])
    plt.show 
    guassianBlur = cv2.GaussianBlur(gray,(5,5),0)
    printImage(guassianBlur,"blurred")
#     cv2.imwrite(name+".png",gray)
cv2.destroyAllWindows()
    

In [ ]:
ret2, th2 = cv2.threshold(grayscaled,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
cv2.imshow('th2',blur)
k1=cv2.waitKey(0)
ed2 = cv2.Canny(th2, 50, 50) 
cv2.imshow('ed2',ed2)
cv2.waitKey()
cv2.destroyAllWindows()